In [1]:
%env CUBLAS_WORKSPACE_CONFIG=:16:8

env: CUBLAS_WORKSPACE_CONFIG=:16:8


In [2]:
from autoencodix.utils.example_data import (
    EXAMPLE_MULTI_BULK,
    EXAMPLE_MULTI_SC,
    EXAMPLE_PROCESSED_DATA,
)

# EXAMPLE_DATA hold PyTorch Datasets (child with extra info) with metdata for train, test and valid splits
processed_data = EXAMPLE_PROCESSED_DATA
raw_bulk = EXAMPLE_MULTI_BULK
raw_sc = EXAMPLE_MULTI_SC

/home/ewald/Github/autoencodix_package/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import autoencodix as acx
from autoencodix.utils.default_config import DefaultConfig

In [4]:

ont_lvl1 = dict()
ont_lvl2 = dict()

ont_lvl1["pwy-1"] = ["sub-pwy-1", "sub-pwy-2"]
ont_lvl1["pwy-2"] = ["sub-pwy-3", "sub-pwy-2"]
# first third of feature ids in processed_data.train.feature_ids
ont_lvl2["sub-pwy-1"] = processed_data.train.feature_ids[: int(len(processed_data.train.feature_ids) / 3)]
# second third of feature ids in processed_data.train.feature_ids
ont_lvl2["sub-pwy-2"] = processed_data.train.feature_ids[
	int(len(processed_data.train.feature_ids) / 3) : int(2 * len(processed_data.train.feature_ids) / 3)
]
# last third of feature ids in processed_data.train.feature_ids
ont_lvl2["sub-pwy-3"] = processed_data.train.feature_ids[
	int(2 * len(processed_data.train.feature_ids) / 3) : int(len(processed_data.train.feature_ids))
]

# ont_lvl2["sub-pwy-1"] = ["gene-1", "gene-2"]
# ont_lvl2["sub-pwy-2"] = ["gene-3", "gene-4"]
# ont_lvl2["sub-pwy-3"] = ["gene-2", "gene-6"]

ontology_tuple = ( ont_lvl1, ont_lvl2 )


In [5]:
# varix = acx.Varix(config=DefaultConfig(), user_data=processed_data)
# varix.preprocess()
# varix.fit()

In [6]:
# varix.preprocessed_data.train.feature_ids

In [7]:
ontix = acx.Ontix(ontologies=ontology_tuple, config=DefaultConfig(), user_data=processed_data)

In [8]:
ontix.preprocess()

In [9]:
len(ont_lvl1.keys())

2

In [10]:
ontology_tuple

({'pwy-1': ['sub-pwy-1', 'sub-pwy-2'], 'pwy-2': ['sub-pwy-3', 'sub-pwy-2']},
 {'sub-pwy-1': ['feature_0',
   'feature_1',
   'feature_2',
   'feature_3',
   'feature_4',
   'feature_5',
   'feature_6',
   'feature_7',
   'feature_8',
   'feature_9'],
  'sub-pwy-2': ['feature_10',
   'feature_11',
   'feature_12',
   'feature_13',
   'feature_14',
   'feature_15',
   'feature_16',
   'feature_17',
   'feature_18',
   'feature_19'],
  'sub-pwy-3': ['feature_20',
   'feature_21',
   'feature_22',
   'feature_23',
   'feature_24',
   'feature_25',
   'feature_26',
   'feature_27',
   'feature_28',
   'feature_29']})

In [11]:
ontix.result.model

Module()

In [12]:
ontix.fit()

cpu not relevant here
Mask layer 0 with shape torch.Size([3, 2]) and 4.0 connections
Mask layer 1 with shape torch.Size([30, 3]) and 30.0 connections
Latent Dim: 2
[30, 7, 2, 2, 2]
Sequential(
  (0): Linear(in_features=2, out_features=3, bias=True)
  (1): Linear(in_features=3, out_features=30, bias=True)
)
torch.Size([32, 2])


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [15]:
# Print all available attributes of ontix
print("Attributes of ontix:")
for attr in dir(ontix):
	if not attr.startswith("__"):
		print(attr)


Attributes of ontix:
_abc_impl
_data_splitter
_dataset_type
_datasets
_evaluator
_fill_data_info
_loss_type
_model_type
_preprocessor
_preprocessor_type
_trainer_type
_validate_config_data
_validate_container
_validate_raw_user_data
_validate_user_data
_visualizer
config
evaluate
fit
ontologies
predict
preprocess
preprocessed_data
raw_user_data
result
run
sample_latent_space
show_result
visualize
